## Writing a custom decision tree
source: https://github.com/random-forests/tutorials/blob/master/decision_tree.py

Youtube: https://www.youtube.com/watch?v=LDRbO9a6XPU&list=PLOU2XLYxmsIIuiBfYad6rFYQU_jL2ryal&index=8

In [1]:
from __future__ import print_function

In [2]:
#toy dataset (2d list)
#the last column is the label--> fruit
#first two columns are features--> color and size
training_data = [
    ['green', 3, 'apple'],
    ['yellow', 3, 'apple'],
    ['red', 1, 'grape'],
    ['red', 1, 'grape'],
    ['yellow', 3, 'lemon']
]

testing_data = [
    ['green', 3, 'apple'],
    ['yellow', 4, 'apple'],
    ['red', 2, 'grape'],
    ['red', 1, 'grape'],
    ['yellow', 3, 'lemon']
]

In [3]:
#adding column header (only for printing trees, not needed for taining)
header = ['color', 'diameter', 'label']

In [4]:
#defining utility function - 1
def unique_vals(dataset, col):
    #find the unique values for a column in a dataset
    return set([row[col] for row in dataset])
#example
unique_vals(training_data,0)

{'green', 'red', 'yellow'}

In [5]:
#defining utility function - 2
def class_counts(dataset):
    #counts the number of each type of example in a dataset
    #return the frequency of labels
    counts = {} #empty dict of label--> count
    for row in dataset:
        label = row[-1]
        if label not in counts:
            counts[label] =0 #creating dict entry for the first time
        counts[label]+=1
    return counts
#example
class_counts(training_data)

{'apple': 2, 'grape': 2, 'lemon': 1}

In [6]:
#defining utility function - 3
def is_numeric(value):
    #test if a value is numeric
    return isinstance(value,int) or isinstance(value,float)
#example
is_numeric(7)

True

In [8]:
#defining question class for the decision tree
#use to populate set of questions/conditions
class Question:
    #question is asked to partition the data set in the tree
    def __init__(self, column, value):
        self.column = column
        self.value = value
    def match(self, example): #takes training set row
        #compare the feature val in the example to that in the question
        val = example[self.column]
        if is_numeric(val):
            return val>=self.value #returns a boolean
        else:
            return val == self.value #returns a boolean
    def __repr__(self):
        #helper method to print
        #print question in a readable format
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
            return "Is %s %s %s?" %(
            header[self.column], condition, str(self.value))
#example
Question(1,3)
q = Question(0,'green')
q.match(training_data[0])

True

In [9]:
#defining utility function - 4
#partitions a dataset based on the question
#For each row in the dataset, check if it matches the question. 
#If so, add it to 'true rows', otherwise, add it to 'false rows'.

def partition(dataset, question):
    true_rows, false_rows = [], []
    for row in dataset:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows
#demo
partition(training_data, Question(0, 'red'))

([['red', 1, 'grape'], ['red', 1, 'grape']],
 [['green', 3, 'apple'], ['yellow', 3, 'apple'], ['yellow', 3, 'lemon']])

In [10]:
#calculat the Gini impurity for a list of rows
#There are a few different ways to do this, I thought this one was
#the most concise. See:
#https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
def gini(dataset):
    counts = class_counts(dataset)
    impurity = 1
    for label in counts:
        prob_of_label = counts[label]/float(len(dataset))
        impurity -= prob_of_label**2
    return impurity
#demo
no_mixing = [['apple'], ['apple']]
gini(no_mixing)
mixing = [['apple'], ['grape']]
gini(mixing)

0.5

In [11]:
"""
calculate information gain --> The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
"""
def info_gain(left, right, current_uncertainty):
    p = float(len(left))/ (len(left)+len(right))
    return current_uncertainty -p * gini(left) - (1-p)*gini(right)

#demo
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0,'red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [12]:
#find the best question to ask by iterating over every feature/value
#and calculating information gain
def find_best_split(dataset):
    best_gain = 0 #keep track of best information gain
    best_question = None #keep track of best question to produce best_gain
    current_uncertainty = gini(dataset)
    n_features = len(dataset[0])-1
    for col in range(n_features):
        values = set(row[col] for row in dataset) #unique feature values
        for val in values:
            question = Question(col, val) #create question
            #try spliting the data
            true_rows, false_rows = partition(dataset, question)
            #skip the split if partition cannot be done
            if len(true_rows)==0 or len(false_rows) == 0:
                continue
            #calculate info gain
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain>=best_gain:
                best_gain, best_question = gain, question
    return best_gain, best_question

#demo
best_gain, best_question = find_best_split(training_data)
print(f"Best gain and question: {best_gain} , {best_question}")

Best gain and question: 0.37333333333333324 , Is diameter >= 3?


## defining several aspects before building the tree

In [13]:
#the leaf node classifies data
#this holds a dict of class -> number of times it appears in the rows
#from the training data that reach this leaf
class leaf:
    def __init__(self, dataset):
        self.predictions = class_counts(dataset)
#A decision node -> Holds the reference to a question and to the
#two child nodes
class Decision_Node:
    def __init__(self,
                question,
                true_branch,
                false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        

In [14]:
#build the tree
#build the recursion. base case -> no further information gain
def build_tree(dataset):
    best_gain, best_question = find_best_split(dataset)
    #base case definition
    if best_gain == 0:
        return leaf(dataset)
    true_rows, false_rows = partition(dataset, best_question)
    #build recursion
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(best_question, true_branch, false_branch) 

In [15]:
#printing tree
def print_tree(node, spacing =""):
    #base case: in case we have a leaf
    if isinstance(node, leaf):
        print(spacing+"Predict", node.predictions)
        return
    print (spacing+str(node.question))
    #call this function recursively over the branches
    print(spacing+'--> True:')
    print_tree(node.true_branch, spacing+" ")
    print_tree(node.false_branch, spacing+" ")

In [20]:
#write a classifer: trained by build_tree
def classify(row, node):
    #base case: in case leaf node 
    if isinstance(node, leaf):
        return node.predictions
    #decide whether to follow true/false brance
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [21]:
my_tree = build_tree(training_data)
classify(training_data[0], my_tree)
#print_tree(my_tree)

{'apple': 1}